In [1]:
import pandas as pd
from datetime import datetime

# Original data
data = {
  "persons": [
    {
      "id": "person1",
      "confidence": 0.95,
      "bbox": [35.7796, -78.6382, 0, 0],
      "image": "base64_encoded_image_data_here",
      "timestamp": "2023-05-20T14:30:00Z"
    },
    {
      "id": "person2",
      "confidence": 0.88,
      "bbox": [35.7798, -78.6385, 0, 0],
      "image": "base64_encoded_image_data_here",
      "timestamp": "2023-05-20T14:31:00Z"
    },
    {
      "id": "person3",
      "confidence": 0.92,
      "bbox": [35.7794, -78.6380, 0, 0],
      "image": "base64_encoded_image_data_here",
      "timestamp": "2023-05-20T14:32:00Z"
    }
  ],
  "hazards": [
    {
      "id": "hazard1",
      "type": "warning",
      "location": { "lat": 35.7800, "lng": -78.6390 },
      "severity": "Moderate",
      "details": "Potential structural damage detected",
      "createdBy": "AI System",
      "createdAt": "2023-05-20T14:25:00Z"
    },
    {
      "id": "hazard2",
      "type": "fire",
      "location": { "lat": 35.7792, "lng": -78.6375 },
      "severity": "High",
      "details": "Active fire detected in residential area",
      "createdBy": "Thermal Sensor",
      "createdAt": "2023-05-20T14:28:00Z"
    }
  ],
  "drones": [
    {
      "name": "Drone X123",
      "isConnected": True,
      "batteryLevel": 85,
      "location": { "lat": 35.7796, "lng": -78.6382 },
      "startingCoordinate": "35.7796, -78.6382",
      "timestamp": "2023-05-20T14:30:00Z"
    },
    {
      "name": "Drone Y456",
      "isConnected": True,
      "batteryLevel": 72,
      "location": { "lat": 35.7798, "lng": -78.6385 },
      "startingCoordinate": "35.7798, -78.6385",
      "timestamp": "2023-05-20T14:30:00Z"
    },
    {
      "name": "Drone Z789",
      "isConnected": True,
      "batteryLevel": 93,
      "location": { "lat": 35.7794, "lng": -78.6380 },
      "startingCoordinate": "35.7794, -78.6380",
      "timestamp": "2023-05-20T14:30:00Z"
    }
  ]
}

# Convert persons data
persons_df = pd.DataFrame([
    {
        "id": person["id"],
        "location_lat": person["bbox"][0],
        "location_lng": person["bbox"][1],
        "timestamp": datetime.strptime(person["timestamp"], '%Y-%m-%dT%H:%M:%SZ')
    } for person in data["persons"]
])

# Convert drones data
drones_df = pd.DataFrame([
    {
        "name": drone["name"],
        "is_connected": drone["isConnected"],
        "battery_level": drone["batteryLevel"],
        "location_lat": drone["location"]["lat"],
        "location_lng": drone["location"]["lng"],
        "timestamp": datetime.strptime(drone["timestamp"], '%Y-%m-%dT%H:%M:%SZ')
    } for drone in data["drones"]
])

# Convert hazards data
hazards_df = pd.DataFrame([
    {
        "id": hazard["id"],
        "type": hazard["type"],
        "location_lat": hazard["location"]["lat"],
        "location_lng": hazard["location"]["lng"],
        "severity": hazard["severity"],
        "details": hazard["details"],
        "created_by": hazard["createdBy"],
        "created_at": datetime.strptime(hazard["createdAt"], '%Y-%m-%dT%H:%M:%SZ')
    } for hazard in data["hazards"]
])

# Export to parquet files
persons_df.to_parquet("persons.parquet", index=False)
drones_df.to_parquet("drones.parquet", index=False)
hazards_df.to_parquet("hazards.parquet", index=False)


In [10]:
!pip install python-dotenv
!pip install singlestoredb

  Using cached singlestoredb-1.7.2-cp38-abi3-macosx_10_9_universal2.whl.metadata (5.6 kB)
  Using cached PyJWT-2.9.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached build-1.2.2.post1-py3-none-any.whl.metadata (6.5 kB)
  Using cached parsimonious-0.10.0-py3-none-any.whl.metadata (25 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached sqlparams-6.1.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached pyproject_hooks-1.2.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached urllib3-2.2.3-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
Using cached singlestoredb-1.7.2-cp38-abi3-macosx_10_9_universal2.whl (357 kB)
Using cached build-1.2.2.post1-py3-none-any.whl (22 kB)
Using cached parsimonious-0.10.0-py3-none-any.whl (48 kB)
Using cached PyJWT-2.9.0-py3-none-any.whl (22 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached sqlparams-6.1.0-py3-none-any.whl (17 kB)
Using cached certifi-202

In [11]:
import os
from dotenv import load_dotenv
import singlestoredb
load_dotenv()
# SingleStore connection
conn = singlestoredb.connect(
    host=os.getenv('SINGLESTORE_HOST'),
    port=int(os.getenv('SINGLESTORE_PORT')),
    user=os.getenv('SINGLESTORE_USER'),
    password=os.getenv('SINGLESTORE_PASSWORD'),
    database=os.getenv('SINGLESTORE_DATABASE')
)


In [12]:
# Load parquet files into SingleStore database

# Load persons data
with conn.cursor() as cursor:
    cursor.execute("""
    LOAD DATA INFILE 'persons.parquet'
    INTO TABLE persons
    (
        location_lat <- location_lat,
        location_lng <- location_lng,
        timestamp <- timestamp
    )
    FORMAT PARQUET;
    """)


OperationalError: 1054: Unknown column 'location_lat' in 'field list'

In [ ]:
# Load drones data
with conn.cursor() as cursor:
    cursor.execute("""
    LOAD DATA INFILE 'drones.parquet'
    INTO TABLE drone_status
    (
        name <- name,
        is_connected <-  is_connected,
        battery_level <- battery_level,  
        location_lat <- location_lat,
        location_lng <- location_lng,
        timestamp <- timestamp
    )
    FORMAT PARQUET;
    """)

In [ ]:
# Load hazards data
with conn.cursor() as cursor:
    cursor.execute("""
    LOAD DATA INFILE 'hazards.parquet'
    INTO TABLE hazards
    (
        type <- type,
        location_lat <- location_lat,
        location_lng <- location_lng,
        severity <- severity,
        details <- details,
        created_by <- created_by,
        created_at <- created_at
    )
    FORMAT PARQUET;
    """)

print("Data loaded successfully into SingleStore database.")